1. 데이터 정보 확인하기
2. 데이터 전처리
    - 불필요한 열 제거
    - 결측치 제거
    - 이상치 제거
    - 데이터 스케일링
    - 원핫인코딩
    - 필요시 파생변수 생성
3. train_test_set 분리 (되어있는 경우가 많음)
4. 모델 평가

In [106]:
import numpy as np
import pandas as pd

X_test = pd.read_csv('./data/titanic_x_test.csv', encoding = 'cp949')
X_train = pd.read_csv('./data/titanic_x_train.csv', encoding = 'cp949')
y_train = pd.read_csv('./data/titanic_y_train.csv', encoding = 'cp949')

X_test.shape, X_train.shape, y_train.shape

x_test_id = X_test.PassengerId

In [107]:
X_train = X_train.drop(columns = ['PassengerId'])
y_train = y_train.drop(columns = ['癤풮assengerId'])
X_test = X_test.drop(columns = ['PassengerId'])

In [108]:
df = pd.concat([X_train, y_train], axis=1)
df[['나이', 'Survived']].corr() # 나이 t삭제

,나이,Survived
나이,1.000000,-0.077221
Survived,-0.077221,1.000000


In [109]:
X_train = X_train.drop(columns = ['나이', '객실번호', '승객이름', '티켓번호'])
X_test = X_test.drop(columns = ['나이', '객실번호', '승객이름', '티켓번호'])

In [110]:
X_train['선착장'].value_counts()

S    644
C    168
Q     77
Name: 선착장, dtype: int64

In [111]:
X_test['선착장'].value_counts()

S    270
C    102
Q     46
Name: 선착장, dtype: int64

In [112]:
X_train['선착장'] = X_train['선착장'].fillna('S')

In [113]:
X_test['운임요금'] = X_test['운임요금'].fillna(round(X_test['운임요금'].mean(),4))

In [114]:
print(X_train.head(3))

   티켓등급      성별  형제자매배우자수  부모자식수     운임요금 선착장
0     3    male         1      0   7.2500   S
1     1  female         1      0  71.2833   C
2     3  female         0      0   7.9250   S


In [115]:
X_train['가족수'] = X_train['형제자매배우자수'] + X_train['부모자식수']
X_test['가족수'] = X_test['형제자매배우자수'] + X_test['부모자식수']

In [116]:
X_train = X_train.drop(columns = ['형제자매배우자수', '부모자식수'])
X_test = X_test.drop(columns = ['형제자매배우자수', '부모자식수'])

In [117]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

X_train = pd.concat([X_train, pd.DataFrame(OneHotEncoder(sparse=False).fit_transform(X_train[['선착장']]))], axis = 1)
X_train.head()

,티켓등급,성별,운임요금,선착장,가족수,0,1,2
0,3,male,7.2500,S,1,0.0,0.0,1.0
1,1,female,71.2833,C,1,1.0,0.0,0.0
2,3,female,7.9250,S,0,0.0,0.0,1.0
3,1,female,53.1000,S,1,0.0,0.0,1.0
4,3,male,8.0500,S,0,0.0,0.0,1.0


In [118]:
X_test = pd.concat([X_test, pd.DataFrame(OneHotEncoder(sparse=False).fit_transform(X_test[['선착장']]))], axis = 1)
X_test.head()

,티켓등급,성별,운임요금,선착장,가족수,0,1,2
0,3,male,7.8292,Q,0,0.0,1.0,0.0
1,3,female,7.0000,S,1,0.0,0.0,1.0
2,2,male,9.6875,Q,0,0.0,1.0,0.0
3,3,male,8.6625,S,0,0.0,0.0,1.0
4,3,female,12.2875,S,2,0.0,0.0,1.0


In [119]:
X_train = X_train.drop(columns = ['선착장'])
X_test = X_test.drop(columns = ['선착장'])

In [120]:
from sklearn.preprocessing import MinMaxScaler

X_train['운임요금'] = MinMaxScaler().fit_transform(X_train[['운임요금']])
X_test['운임요금'] = MinMaxScaler().fit_transform(X_test[['운임요금']])
X_train.head()

,티켓등급,성별,운임요금,가족수,0,1,2
0,3,male,0.014151,1,0.0,0.0,1.0
1,1,female,0.139136,1,1.0,0.0,0.0
2,3,female,0.015469,0,0.0,0.0,1.0
3,1,female,0.103644,1,0.0,0.0,1.0
4,3,male,0.015713,0,0.0,0.0,1.0


In [121]:
X_train = pd.concat([X_train, pd.DataFrame(OneHotEncoder(sparse=False).fit_transform(X_train[['성별']]), columns=['male', 'female'])], axis = 1)
X_test = pd.concat([X_test, pd.DataFrame(OneHotEncoder(sparse=False).fit_transform(X_test[['성별']]), columns=['male', 'female'])], axis = 1)
X_train.head()

,티켓등급,성별,운임요금,가족수,0,1,2,male,female
0,3,male,0.014151,1,0.0,0.0,1.0,0.0,1.0
1,1,female,0.139136,1,1.0,0.0,0.0,1.0,0.0
2,3,female,0.015469,0,0.0,0.0,1.0,1.0,0.0
3,1,female,0.103644,1,0.0,0.0,1.0,1.0,0.0
4,3,male,0.015713,0,0.0,0.0,1.0,0.0,1.0


In [122]:
X_train = X_train.drop(columns = ['성별'])
X_test = X_test.drop(columns = ['성별'])

In [125]:
# 모델 학습
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb = XGBClassifier()
xgb.fit(X_train, y_train)

[21:06:04] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [126]:
y_test_predict = pd.DataFrame(xgb.predict(X_test))
y_test_predict.head()

,0
0,0
1,1
2,0
3,0
4,1


In [128]:
xgb.score(X_test, y_test_predict)

1.0

In [137]:
df2 = pd.concat([x_test_id, y_test_predict], axis = 1)
df2.columns =  [['PassengerId', 'Survived']]
df2.to_csv('000000.csv', index = False)